# ch1. subword tokenization

이번 챕터에서는 subword 기반의 토큰화 기법에 대해서 알아보겠습니다. 지금까지 사용해본 토큰화 기법은 다음과 같은 종류가 있었습니다.

- 공백 기준 토큰화
- 단어 기준 토큰화
- 형태소 기준 토큰화

그러나 이는 신조어나 고유 명사들에 취약한 약점이 있었습니다. 이를 극복하기 위해 단어를 더 작은 단위인 subword로 나누어 토큰화 한 것이 subword tokenizer입니다.

In [1]:
!pip install tokenizers

DEPRECATION: pytorch-lightning 1.8.3.post1 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: mecab-python 0.996-ko-0.9.2 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


## Byte Pair Encoding


### Byte Pair Encoding 개념
BPE라고 줄여서 많이 표기합니다. 본래 데이터 압축 알고리즘으로, 데이터에서 가장 많이 등장한 문자열을 병합해서 압축하는 기법입니다.

- aaabdaaabac

위와 같은 문자열이 주어졌다고 가정하겠습니다. 단어에 등장한 글자로 사전을 구성하면 (a, b, c, d)가 됩니다. 이제 가장 많이 등장하는 연속하는 두 글자를 하나로 병합합니다. 여기서는 aa가 동시에 많이 등장하므로 이를 Z로 치환해보겠습니다.

- ZabdZabac

이제 글자 사전은 (a, b, c, d, Z)가 됩니다. 이 문자열에서 다시 ab를 Y로 압축해보겠습니다.

- ZYdZYac

사전에는 Y가 추가되어서 (a, b, c, d, Z, Y)가 됩니다. 마지막으로 ZY를 묶어서 X로 치환해보겠습니다.

- XdXac

최종 사전은 (a, b, c, d, Z, Y, X)가 됩니다. 정리해보면 글자 사전은 처음 4개에서 7개로 어휘가 3개 증가했지만, 문자열은 처음 길이 11에서 5로 줄었습니다. 즉, BPE는 사전 크기 증가는 억제하면서 정보를 효율적으로 압축할 수 있는 알고리즘입니다. 

BPE의 가장 큰 장점은 분석 대상 언어에 대한 사전 지식이 필요 없습니다. 그저 말뭉치 데이터만 주어지면 자주 나타나는 문자열을 토큰으로 분석하기 때문에 어떤 언어에도 적용이 가능합니다.

### BPE 토큰화 과정

1. 어휘 집합 구축

자주 등장하는 문자열을 병합하고 이를 어휘 집합에 추가합니다. 미리 지정한 어휘 집합의 크기가 될 때까지 반복합니다.

2. 토큰화

토큰화 대상 문장을 띄어쓰기로 분리하고, 어휘 집합에 있는 subword가 포함되어 있을 때 해당 subword를 각 어절에서 분리합니다.

## BPE 학습시키기 

먼저 아래와 같이 말뭉치 데이터가 주어졌다고 가정하겠습니다.

In [2]:
corpus = [
    "this is the hugging face course .",
    "this chapter is about tokenization .",
    "this section shows several tokenizer algorithms .",
    "Hopefully, you will be able to understand how they are trained and generate tokens."
]

### Pre-tokenize & build vocab

이를 공백을 기준으로 나눠준 뒤, 처음 vocab 사전을 만들어보겠습니다.

In [3]:
words = []
for sentence in corpus:
    words.extend(sentence.split(" "))

In [4]:
print(words)

['this', 'is', 'the', 'hugging', 'face', 'course', '.', 'this', 'chapter', 'is', 'about', 'tokenization', '.', 'this', 'section', 'shows', 'several', 'tokenizer', 'algorithms', '.', 'Hopefully,', 'you', 'will', 'be', 'able', 'to', 'understand', 'how', 'they', 'are', 'trained', 'and', 'generate', 'tokens.']


In [5]:
vocabs = set()
for word in words:
    for c in word:
        vocabs.add(c)

In [6]:
print(vocabs)

{'a', 'o', 'v', ',', 'w', 's', 'e', 'r', 'd', '.', 'c', 'n', 'u', 'z', 't', 'f', 'g', 'm', 'p', 'h', 'y', 'b', 'k', 'i', 'l', 'H'}


### tokenize using vocab 

사전을 이용하여 각 단어들을 토큰화 하는 함수를 작성해보겠습니다. 이때, 단어를 vocab에 들어있는 토큰 단위로 쪼개기 위해서 정규표현식을 사용하겠습니다.

In [7]:
import re

def build_pattern(vocabs):
    sorted_vocabs = sorted(vocabs, key=lambda x: len(x), reverse=True)
    pattern = re.compile(f"|".join(sorted_vocabs))
    return pattern

In [8]:
pattern = build_pattern(vocabs)
print(pattern)

re.compile('a|o|v|,|w|s|e|r|d|.|c|n|u|z|t|f|g|m|p|h|y|b|k|i|l|H')


In [9]:
def tokenize(words, pattern):
    tokenized = []
    for word in words:
        tokens = pattern.findall(word)
        tokenized.append(tokens)
    return tokenized

In [10]:
tokenize(words, pattern)

[['t', 'h', 'i', 's'],
 ['i', 's'],
 ['t', 'h', 'e'],
 ['h', 'u', 'g', 'g', 'i', 'n', 'g'],
 ['f', 'a', 'c', 'e'],
 ['c', 'o', 'u', 'r', 's', 'e'],
 ['.'],
 ['t', 'h', 'i', 's'],
 ['c', 'h', 'a', 'p', 't', 'e', 'r'],
 ['i', 's'],
 ['a', 'b', 'o', 'u', 't'],
 ['t', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n'],
 ['.'],
 ['t', 'h', 'i', 's'],
 ['s', 'e', 'c', 't', 'i', 'o', 'n'],
 ['s', 'h', 'o', 'w', 's'],
 ['s', 'e', 'v', 'e', 'r', 'a', 'l'],
 ['t', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r'],
 ['a', 'l', 'g', 'o', 'r', 'i', 't', 'h', 'm', 's'],
 ['.'],
 ['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y', ','],
 ['y', 'o', 'u'],
 ['w', 'i', 'l', 'l'],
 ['b', 'e'],
 ['a', 'b', 'l', 'e'],
 ['t', 'o'],
 ['u', 'n', 'd', 'e', 'r', 's', 't', 'a', 'n', 'd'],
 ['h', 'o', 'w'],
 ['t', 'h', 'e', 'y'],
 ['a', 'r', 'e'],
 ['t', 'r', 'a', 'i', 'n', 'e', 'd'],
 ['a', 'n', 'd'],
 ['g', 'e', 'n', 'e', 'r', 'a', 't', 'e'],
 ['t', 'o', 'k', 'e', 'n', 's', '.']]

### Pair counting 
이제 어휘를 추가하기 위해서 함께 자주 등장하는 쌍을 찾아보겠습니다.

In [11]:
from collections import Counter 

def count_pairs(words, pattern):
    c = Counter()
    for word in words:
        tokens = pattern.findall(word)
        for i in range(len(tokens) - 1):
            c[f"{tokens[i]}{tokens[i+1]}"] += 1
    return c

In [12]:
pair_counter = count_pairs(words, pattern)

In [13]:
print(pair_counter)

Counter({'th': 6, 'is': 5, 'er': 5, 'to': 4, 'en': 4, 'hi': 3, 'ou': 3, 'se': 3, 'ok': 3, 'ke': 3, 'ra': 3, 'nd': 3, 'he': 2, 'in': 2, 'rs': 2, 'te': 2, 'ab': 2, 'ni': 2, 'iz': 2, 'at': 2, 'ti': 2, 'io': 2, 'on': 2, 'ho': 2, 'ow': 2, 'al': 2, 'll': 2, 'an': 2, 'ne': 2, 'hu': 1, 'ug': 1, 'gg': 1, 'gi': 1, 'ng': 1, 'fa': 1, 'ac': 1, 'ce': 1, 'co': 1, 'ur': 1, 'ch': 1, 'ha': 1, 'ap': 1, 'pt': 1, 'bo': 1, 'ut': 1, 'za': 1, 'ec': 1, 'ct': 1, 'sh': 1, 'ws': 1, 'ev': 1, 've': 1, 'ze': 1, 'lg': 1, 'go': 1, 'or': 1, 'ri': 1, 'it': 1, 'hm': 1, 'ms': 1, 'Ho': 1, 'op': 1, 'pe': 1, 'ef': 1, 'fu': 1, 'ul': 1, 'ly': 1, 'y,': 1, 'yo': 1, 'wi': 1, 'il': 1, 'be': 1, 'bl': 1, 'le': 1, 'un': 1, 'de': 1, 'st': 1, 'ta': 1, 'ey': 1, 'ar': 1, 're': 1, 'tr': 1, 'ai': 1, 'ed': 1, 'ge': 1, 'ns': 1, 's.': 1})


### vocab 추가

가장 많이 등장한 쌍을 사전에 추가하겠습니다. 그 다음 words를 다시 토큰화 하여 결과를 확인해보겠습니다.

In [14]:
most_common_pair = pair_counter.most_common()[0][0]
vocabs.add(most_common_pair)
pattern = build_pattern(vocabs)
tokenize(words, pattern)

[['th', 'i', 's'],
 ['i', 's'],
 ['th', 'e'],
 ['h', 'u', 'g', 'g', 'i', 'n', 'g'],
 ['f', 'a', 'c', 'e'],
 ['c', 'o', 'u', 'r', 's', 'e'],
 ['.'],
 ['th', 'i', 's'],
 ['c', 'h', 'a', 'p', 't', 'e', 'r'],
 ['i', 's'],
 ['a', 'b', 'o', 'u', 't'],
 ['t', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n'],
 ['.'],
 ['th', 'i', 's'],
 ['s', 'e', 'c', 't', 'i', 'o', 'n'],
 ['s', 'h', 'o', 'w', 's'],
 ['s', 'e', 'v', 'e', 'r', 'a', 'l'],
 ['t', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r'],
 ['a', 'l', 'g', 'o', 'r', 'i', 'th', 'm', 's'],
 ['.'],
 ['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y', ','],
 ['y', 'o', 'u'],
 ['w', 'i', 'l', 'l'],
 ['b', 'e'],
 ['a', 'b', 'l', 'e'],
 ['t', 'o'],
 ['u', 'n', 'd', 'e', 'r', 's', 't', 'a', 'n', 'd'],
 ['h', 'o', 'w'],
 ['th', 'e', 'y'],
 ['a', 'r', 'e'],
 ['t', 'r', 'a', 'i', 'n', 'e', 'd'],
 ['a', 'n', 'd'],
 ['g', 'e', 'n', 'e', 'r', 'a', 't', 'e'],
 ['t', 'o', 'k', 'e', 'n', 's', '.']]

### 지정한 vocab 수 달성까지 반복 

미리 지정한 vocab 수를 달성할 때까지 반복해줍니다. vocab 수가 50이 될 때까지 반복한 뒤에 토큰화 해보겠습니다. 

In [15]:
vocab_size = 50
while len(vocabs) < vocab_size:
    print(f"len vocabs: {len(vocabs)}")
    print(vocabs)
    pattern = build_pattern(vocabs)
    pair_counter = count_pairs(words, pattern)
    most_common_pair = pair_counter.most_common()[0][0]
    vocabs.add(most_common_pair)

len vocabs: 27
{'a', 'o', 'v', ',', 'w', 's', 'e', 'r', 'd', '.', 'c', 'n', 'u', 'z', 't', 'f', 'g', 'm', 'p', 'h', 'y', 'b', 'k', 'i', 'th', 'l', 'H'}
len vocabs: 28
{'a', 'o', 'v', ',', 'w', 's', 'is', 'e', 'r', 'd', '.', 'c', 'n', 'u', 'z', 't', 'f', 'g', 'm', 'p', 'h', 'y', 'b', 'k', 'i', 'th', 'l', 'H'}
len vocabs: 29
{'a', 'er', 'o', 'v', ',', 'w', 's', 'is', 'e', 'r', 'd', '.', 'c', 'n', 'u', 'z', 't', 'f', 'g', 'm', 'p', 'h', 'y', 'b', 'k', 'i', 'th', 'l', 'H'}
len vocabs: 30
{'a', 'er', 'o', 'v', ',', 'w', 's', 'is', 'e', 'r', 'd', '.', 'to', 'c', 'n', 'u', 'z', 't', 'f', 'g', 'm', 'p', 'h', 'y', 'b', 'k', 'i', 'th', 'l', 'H'}
len vocabs: 31
{'a', 'er', 'o', 'v', ',', 'w', 's', 'is', 'e', 'r', 'd', '.', 'to', 'c', 'n', 'u', 'z', 't', 'f', 'g', 'en', 'm', 'p', 'h', 'y', 'b', 'k', 'i', 'th', 'l', 'H'}
len vocabs: 32
{'a', 'er', 'o', 'v', ',', 'w', 's', 'is', 'e', 'r', 'd', '.', 'to', 'c', 'n', 'u', 'z', 't', 'f', 'g', 'en', 'm', 'p', 'this', 'h', 'y', 'b', 'k', 'i', 'th', 'l', '

반복을 거듭할 수록 같이 자주 등장하는 토큰들이 vocabs에 추가되는 것을 확인할 수 있습니다. 이렇게 얻어낸 vocab으로 토큰화 하는 방식이 BPE tokenizer 입니다.

In [16]:
pattern = build_pattern(vocabs)
tokenize(words, pattern)

[['this'],
 ['is'],
 ['the'],
 ['h', 'u', 'g', 'g', 'in', 'g'],
 ['f', 'a', 'c', 'e'],
 ['c', 'ou', 'r', 'se'],
 ['.'],
 ['this'],
 ['c', 'h', 'a', 'p', 't', 'er'],
 ['is'],
 ['ab', 'ou', 't'],
 ['tokeniz', 'at', 'ion'],
 ['.'],
 ['this'],
 ['se', 'c', 't', 'ion'],
 ['s', 'how', 's'],
 ['se', 'v', 'er', 'al'],
 ['tokeniz', 'er'],
 ['al', 'g', 'o', 'r', 'i', 'th', 'm', 's'],
 ['.'],
 ['H', 'o', 'p', 'e', 'f', 'u', 'll', 'y', ','],
 ['y', 'ou'],
 ['w', 'i', 'll'],
 ['b', 'e'],
 ['ab', 'l', 'e'],
 ['to'],
 ['u', 'nd', 'er', 's', 't', 'and'],
 ['how'],
 ['the', 'y'],
 ['a', 'r', 'e'],
 ['t', 'r', 'a', 'in', 'e', 'd'],
 ['and'],
 ['g', 'en', 'er', 'at', 'e'],
 ['token', 's', '.']]

### OOV 대응

subword 기반의 토큰화 기법들은 신조어나 고유 명사에 강하다고 했습니다. 실제로 학습 과정에서 본 적 없는 단어가 주어졌을 때, 어떻게 토큰화 하는지 살펴보겠습니다.

In [17]:
new_sentence = "tokenization is the first step of natural language processing."
new_words = new_sentence.split(" ")

In [18]:
tokenize(new_words, pattern)

[['tokeniz', 'at', 'ion'],
 ['is'],
 ['the'],
 ['f', 'i', 'r', 's', 't'],
 ['s', 't', 'e', 'p'],
 ['o', 'f'],
 ['n', 'at', 'u', 'r', 'al'],
 ['l', 'a', 'n', 'g', 'u', 'a', 'g', 'e'],
 ['p', 'r', 'o', 'c', 'e', 's', 's', 'in', 'g', '.']]

이런 식으로 BPE tokenization은 한번도 본 적 없는 단어라 할 지라도 subword 단위로 쪼개주는 것을 확인할 수 있습니다.

## 그 외 subword tokenization 기법들

BPE는 가장 기본적인 subword tokenization 기법입니다. 이 외에도 subword tokenization 기법들이 있습니다만, 핵심적인 개념은 비슷하므로 간단히 짚고 넘어가겠습니다.

### WordPiece Tokenizer

WordPiece Tokenizer은 BPE의 변형 알고리즘입니다. BPE가 빈도수에 기반하여 가장 많이 등장한 쌍을 병합하는 것과는 달리, 병합되었을 때 코퍼스의 우도(Likelihood)를 가장 높이는 쌍을 병합합니다. 이 기법은 구글의 language model인 BERT를 학습시킬 때 사용되었습니다.

### Unigram Language Model Tokenizer

유니그램 언어 모델 토크나이저는 각각의 서브워드들에 대해서 손실(loss)을 계산합니다. 여기서 서브 단어의 손실이라는 것은 해당 서브워드가 단어 집합에서 제거되었을 경우, 코퍼스의 우도(Likelihood)가 감소하는 정도를 말합니다. 이렇게 측정된 서브워드들을 손실의 정도로 정렬하여, 최악의 영향을 주는 10~20%의 토큰을 제거합니다. 이를 원하는 단어 집합의 크기에 도달할 때까지 반복합니다.

### 어떤 기법을 선택할 것인가?

subword tokenizer의 경우 corpus를 기반으로 집계한 vocab에 따라서 tokenize 결과가 달라집니다. 때문에 BERT나 llama와 같은 오픈 소스 language model의 경우, 자신들이 사용한 tokenizer를 함께 공개합니다. 이들 모델을 그대로 사용하고 싶거나, fine-tuning 하고 싶다면 이들이 공개한 tokenizer를 사용하시면 됩니다.

그 외에 직접 tokenizer를 학습을 경우, 정답은 없습니다만 WordPiece Tokenizer가 선호됩니다. vocab size의 경우엔 corpus의 사이즈, 언어, 테스크의 종류에 따라서 달라질 수 있습니다. 작게 시작하고 서서히 늘려보는 것을 추천합니다. 

subword tokenizer에 대해서 추가적으로 궁금하신 분들은 아래 논문을 참고해주세요.  
A COMPREHENSIVE ANALYSIS OF SUBWORD TOKENIZERS FOR
MORPHOLOGICALLY RICH LANGUAGES: https://www.cmpe.boun.edu.tr/~gungort/theses/A%20Comprehensive%20Analysis%20of%20Subword%20Tokenizers%20for%20Morphologically%20Rich%20Languages.pdf

## 정리

이번 챕터에서는 subword tokenization의 핵심 개념과 주요 기법들에 대해서 알아보았습니다. 다음 챕터에서는 huggingface 라이브러리를 이용해서 직접 subword tokenizer를 학습시키고 사용하는 방법에 대해서 알아보겠습니다.